In [5]:
# importing libraries
import requests
import json 
import pandas as pd
import hvplot.pandas

# pulling API data from RedFin
url = "https://unofficial-redfin.p.rapidapi.com/properties/list"

querystring = {"region_id":"14240","region_type":"6","uipt":"1,2,3,7,8","status":"9","sf":"1,2,5,6,7","num_homes":"300","sold_within_days":"30"}

headers = {
	"X-RapidAPI-Key": "b6a21fcaa0msh2e9de61afa99433p14003ajsnd33bf08e42ad",
	"X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

dataPHX = response.json()

# display API pull status
print(f"Start API acquisition")
print(f"------------&&&&-------------------")

print(json.dumps(dataPHX, indent=4))

querystring = {"region_id":"7102","region_type":"6","uipt":"1,2,3,7,8","status":"9","sf":"1,2,5,6,7","num_homes":"300","sold_within_days":"30"}

headers = {
	"X-RapidAPI-Key": "b6a21fcaa0msh2e9de61afa99433p14003ajsnd33bf08e42ad",
	"X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

dataGLEN = response.json()

print(json.dumps(dataGLEN, indent=4))

Start API acquisition
------------&&&&-------------------
{
    "homes": [
        {
            "homeData": {
                "propertyId": "27130840",
                "listingId": {
                    "value": "151594833"
                },
                "listingDisplayLevel": "ACCESSIBLE",
                "mlsId": "6414207",
                "url": "/AZ/Phoenix/11640-N-Tatum-Blvd-85028/unit-2054/home/27130840",
                "dataSourceId": {
                    "value": "86"
                },
                "marketId": {
                    "value": "11"
                },
                "mlsStatusId": {
                    "value": "292"
                },
                "servicePolicyId": {
                    "value": "4"
                },
                "listingMetadata": {
                    "searchStatus": "SOLD",
                    "listingType": "MLS",
                    "isRedfin": true,
                    "hasVirtualTour": true
                },
           

In [8]:
property_id = []
property_type = []
beds = []
baths = []
square_ft_info = []
price_info = []
latitude = [] 
longitude = [] 

for homeData in dataGLEN['homes']:
    property_id.append(homeData['homeData']['propertyId'])
    property_type.append(homeData['homeData']['propertyType'])
    #beds.append(homeData['homeData']['beds']['value'])
    #baths.append(homeData['homeData']['baths']['value'])
    try:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']['value'])
    except KeyError:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']["NA"])
    price_info.append(homeData['homeData']['priceInfo']['amount']['value'])
    latitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['latitude'])
    longitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['longitude'])

for homeData in dataPHX['homes']:
    property_id.append(homeData['homeData']['propertyId'])
    property_type.append(homeData['homeData']['propertyType'])
    #beds.append(homeData['homeData']['beds']['value'])
    #baths.append(homeData['homeData']['baths']['value'])
    try:
        square_ft_info.append(homeData['homeData']['sqftInfo']['amount']['value'])
    except KeyError:
        square_ft_info.append("0")
    price_info.append(homeData['homeData']['priceInfo']['amount']['value'])
    latitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['latitude'])
    longitude.append(homeData['homeData']['addressInfo']['centroid']['centroid']['longitude'])

In [11]:
data_dict = {"Property_ID": property_id, "Property_Type": property_type, "Square Footage": square_ft_info, "Price Info": price_info, "Latitude": latitude, "Longitude": longitude}
data_df = pd.DataFrame(data_dict)
data_df['Square Footage']=data_df['Square Footage'].astype(float)
data_df['Price Info']=data_df['Price Info'].astype(float)
data_df['PPSF'] = data_df['Price Info'] / data_df['Square Footage']
data_df

,Property_ID,Property_Type,Square Footage,Price Info,Latitude,Longitude,PPSF
0,27760014,SINGLE_FAMILY_RESIDENTIAL,1706.0,440000.0,33.511944,-112.189664,257.913247
1,27203573,SINGLE_FAMILY_RESIDENTIAL,1139.0,376000.0,33.590093,-112.172200,330.114135
2,26911516,TOWNHOUSE,1576.0,274900.0,33.531714,-112.152392,174.428934
3,28384115,CONDO_COOP,756.0,185000.0,33.531942,-112.161606,244.708995
4,28135621,SINGLE_FAMILY_RESIDENTIAL,1340.0,410000.0,33.633657,-112.165447,305.970149
...,...,...,...,...,...,...,...
595,27438658,SINGLE_FAMILY_RESIDENTIAL,2401.0,450000.0,33.421813,-112.281410,187.421908
596,28100174,SINGLE_FAMILY_RESIDENTIAL,2874.0,444700.0,33.416902,-112.266833,154.732081
597,27757289,SINGLE_FAMILY_RESIDENTIAL,1674.0,399995.0,33.422038,-112.276240,238.945639
598,28422105,SINGLE_FAMILY_RESIDENTIAL,1366.0,385000.0,33.425794,-112.274191,281.844802


In [12]:
data_df.hvplot.points(
    'Longitude',
    'Latitude', 
    geo = True, 
    frame_width = 700, 
    frame_height = 500, 
    tiles = 'OSM',
    color = 'Property_Type',
    size = 'PPSF',
    hover_cols = 'Price Info',
    title = 'Phoenix and Glendale Housing Property Type Listings'
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Property_Type,PPSF,Price Info)

In [18]:
avg_price = (data_df['Price Info']).mean
print (avg_price)

<bound method NDFrame._add_numeric_operations.<locals>.mean of 0      440000.0
1      376000.0
2      274900.0
3      185000.0
4      410000.0
         ...   
595    450000.0
596    444700.0
597    399995.0
598    385000.0
599    360000.0
Name: Price Info, Length: 600, dtype: float64>
